### 0. Imports

In [2]:
import torch
import torch.nn as nn
import pickle
import numpy as np
from torch.utils.data import Dataset , DataLoader
import sys
#sys.path.append("../snpe/")
#print(sys.path)
#from snpe.simulations import simulator_class, marketplace_simulator_class

### A. Load the data: using simulational output

In [4]:
'''
Loading an example marketplace simulation. 
The idea is to replace this with built in methods from the snpe repo's simulation classes.
'''

with open('sample_simulational_output.pkl', 'rb') as f:
    raw_data = pickle.load(f)
    
'''
Using th function below to find the proudct with the most ratings for training. By now we'll be using just
one product. 
'''
    
def find_it(sim_output):
    '''
    Returns the timeseries histogram of the product with the most reviews within
    the output of a marketplace simulation (timeseries)
    '''
    max_length = []
    for i in range(len(sim_output.get('simulations'))):
        for a in range(len(sim_output.get('simulations')[0])):
            if len(sim_output.get('simulations')[i][a]) > len(max_length):
                max_length = sim_output.get('simulations')[i][a]
    return max_length 

'''
Applying the function and taking a look at the first rows of the result
'''

raw_prod = find_it(raw_data)
print("Sample product has received of a total of " + str(len(raw_prod)) + " reviews")
print("Displaying the ten first elements of the time series:")
print(raw_prod[0:10])
print("Displaying the last ten elements of the time series:")
print(raw_prod[-10:])

Sample product has received of a total of 521 reviews
Displaying the ten first elements of the time series:
[[1. 1. 1. 1. 1.]
 [2. 1. 1. 1. 1.]
 [3. 1. 1. 1. 1.]
 [3. 1. 2. 1. 1.]
 [3. 1. 3. 1. 1.]
 [4. 1. 3. 1. 1.]
 [4. 1. 4. 1. 1.]
 [5. 1. 4. 1. 1.]
 [5. 1. 5. 1. 1.]
 [5. 1. 6. 1. 1.]]
Displaying the last ten elements of the time series:
[[ 52.   6. 430.   1.  27.]
 [ 52.   6. 431.   1.  27.]
 [ 53.   6. 431.   1.  27.]
 [ 53.   6. 432.   1.  27.]
 [ 53.   6. 433.   1.  27.]
 [ 53.   6. 434.   1.  27.]
 [ 53.   6. 435.   1.  27.]
 [ 53.   6. 436.   1.  27.]
 [ 53.   6. 437.   1.  27.]
 [ 54.   6. 437.   1.  27.]]


Representing the number of ratings for each value as a proportion of the total (approx): [0.1, 0.01, 0.83, 0.001, 0.05]

### B. Data pre-processing, Dataset and DataLoader

In [3]:
'''
Taking the first differences of the raw data.  
'''

raw_input = [raw_prod[i] - raw_prod[i-1] for i in range(len(raw_prod))]
raw_input = raw_input[1:]
raw_input = np.stack(raw_input, axis=0 )
raw_input

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       ...,
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.]])

A preliminary comment on the foloowing cells below:
In pytorch you can either import pre existing datasets or use your own. To use your own you have to define it as aninstance of the dataset class, including three functions: `__init__, __len__` and `__getitem__` as the guy does below.

- Dataset stores the samples and their corresponding labels, and DataLoader wraps an iterable around the Dataset to enable easy access to the samples.

- read more about this in: 
https://pytorch.org/tutorials/beginner/basics/data_tutorial.html
- Sliding help: https://discuss.pytorch.org/t/dataloader-for-a-lstm-model-with-a-sliding-window/22235 . The SlidingDataset has been defined according to the code proposed by the user in this previous link.

In [4]:
class SlidingDataset(Dataset):
    '''
    This SlidingDataset class is conformed so each input is comprised of the last N observations (from t-n to t) 
    of the series, where n  is the size of the sliding window, and the target is the corresponding observation in 
    t+1.
    '''
    def __init__(self, data, window):
        self.data = data
        #self.data = torch.from_numpy(data)
        self.window = window
        
    def __len__(self): 
        '''
        The __len__ function returns the number of samples in our dataset. In this case intuitively we have
        a number of samples equal to the total number of elements in the time series minus the size of the 
        sliding window. (i.e. How many previous observations conform each input)
        '''
        return len(self.data)-self.window

    def __getitem__(self, index): 
        return  self.data[index : index+self.window], self.data[index+self.window]

In [5]:
'''
Dividing the data into train and test set (Validation is to be added ASAP) and transforming in into tensors
'''
training_size=int(len(raw_input)*0.6)
validation_size=int(len(raw_input)*0.2)
#test_size=int(len(raw_input)*0.2)
train_data, validation, test_data= torch.from_numpy(raw_input[0:training_size,:]),torch.from_numpy(raw_input[training_size:training_size+validation_size,:]), torch.from_numpy(raw_input[training_size+validation_size:len(raw_input),:])

'''
Feeding the tensors to the sliding dataset class. Besides the actual data, the other provided argument is the size
of the sliding windows which in this case is ten.
'''    
train_dataset = SlidingDataset(train_data, 10) # sliding window = 10
test_dataset = SlidingDataset(test_data, 10) # sliding window = 10

'''
The next step is to feed the SlidingDataset objects to the DataLoader class so these can be finally 
provided to the model. In this case these are shuffled and a batch size of 32 observations is defined.
'''
batch_size = 32 
train_dataloader = DataLoader(train_dataset,batch_size, shuffle = True) #,drop_last=True)
test_dataloader = DataLoader(test_dataset,batch_size, shuffle = True)  #,drop_last=True)

### C. A first version of the LSTM model

Definitions of the parametors below taken from pytorch doc. See: https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html
- __Input_dim__: The number of expected features in the input x 
- __Hidden_dim__: The number of features in the hidden state h 
- __Layer_dim__: Number of recurrent layers. E.g., setting num_layers=2 would mean stacking two LSTMs together to form a stacked LSTM, with the second LSTM taking in outputs of the first LSTM and computing the final results. Default: 1



In [6]:
class LSTM_0(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim):
        super(LSTM_0, self).__init__()
        
        self.input_dim = input_dim # n of expected features in the input
        self.hidden_dim = hidden_dim # Hidden dimensions - n of features in hidden state
        self.layer_dim = layer_dim  # Number of hidden layers 
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim) #LSTM
        self.readout = nn.Softmax(dim=0) # Readout layer - translating from hidden space to rating space, dim = dimension that has to add up to 1
        #self.output_layer = nn.Linear(hidden_dim,input_dim)

    def forward(self, x):
        # Initialize hidden state & cell states with zeros 
        # theoretically not 100% necessary - made by default for zeros
        h0 = torch.zeros(self.layer_dim, 10, self.hidden_dim).requires_grad_()
        c0 = torch.zeros(self.layer_dim, 10, self.hidden_dim).requires_grad_()

        out, (hn, cn) = self.lstm(x.double(), (h0.double(),c0.double()))
        #out, (hn, cn) = self.lstm(x)
        final_out = self.readout(out[-1][-1])
 
        return out, hn, cn, final_out

### D. Training loop

In [22]:
def LSTM_train(dataloader):    
    model.train()
    for batch , item in enumerate(dataloader):
        x , y = item
        x = x.to(device)
        y = y.to(device)
        out, hn, cn, final_out = model(x)
        checkme = (out, (hn, cn), final_out)
        ''' Prints for closer inspection '''
        #print("Softmax-processed output -------------")
        #print(checkme[2])
        #print("Last tensor from last cell state (= unprocessed output) -------------")
        #print(out[-1][-1])
        #print("###")
        loss = criterion(out[-1][-1], torch.argmax(y[-1]))
        #loss = criterion(hn[-1][-1], torch.argmax(y[-1]))
        hn = hn.detach() # given the nature of RNN we detach the cn and hn as the are not parameters to be updated (Remove from computational graph)
        cn = cn.detach() # given the nature of RNN we detach the cn and hn as the are not parameters to be updated (Remove from computational graph)
        optimizer.zero_grad() #clears the gradient to 0s
        loss.backward() #calculates the partial derivatives (updates the gradient)
        optimizer.step() #updates the parameters from the gradient
    
        if batch == len(dataloader)-1:
            loss = loss.item()
            print(f"train loss: {loss:>7f} ")

def LSTM_test(dataloader):
    model.eval()
    for batch , item in enumerate(dataloader):
        x , y = item
        x = x.to(device)
        y = y.to(device)
        out, hn, cn, final_out = model(x)
        loss = criterion(out[-1][-1], torch.argmax(y[-1]))
        #loss = criterion(hn[-1][-1], torch.argmax(y[-1]))
        
        if batch == len(dataloader)-1:
            loss = loss.item()
            print(f"test loss: {loss:>7f}")

D.1 -> Brief

In [9]:
'''
Look here -> https://cnvrg.io/pytorch-lstm/
'''
input_dim = 5 
hidden_size = 5
num_layers = 1

device = "cuda" if torch.cuda.is_available() else "cpu"
model = LSTM_0(input_dim , hidden_size , num_layers).to(device)
learning_rate=1e-3

criterion = torch.nn.CrossEntropyLoss()    
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate) 
model = model.double()


num_epochs = 200
for epoch in range(num_epochs): 
    print("epoch " + str(epoch))
    LSTM_train(train_dataloader)
    LSTM_test(test_dataloader)
    print('#####################')    

epoch 0
train loss: 1.310431 
test loss: 1.210353
#####################
epoch 1
train loss: 1.748020 
test loss: 1.157299
#####################
epoch 2
train loss: 1.954507 
test loss: 1.087948
#####################
epoch 3
train loss: 1.048563 
test loss: 1.027377
#####################
epoch 4
train loss: 0.928779 
test loss: 0.917326
#####################
epoch 5
train loss: 1.060510 
test loss: 0.871691
#####################
epoch 6
train loss: 0.831037 
test loss: 0.821833
#####################
epoch 7
train loss: 0.820170 
test loss: 0.775457
#####################
epoch 8
train loss: 0.751845 
test loss: 0.743559
#####################
epoch 9
train loss: 0.716814 
test loss: 0.708380
#####################
epoch 10
train loss: 0.732037 
test loss: 0.694288
#####################
epoch 11
train loss: 0.674424 
test loss: 0.670393
#####################
epoch 12
train loss: 0.681278 
test loss: 0.663717
#####################
epoch 13
train loss: 1.994375 
test loss: 1.923478
##########

train loss: 0.458784 
test loss: 0.457197
#####################
epoch 115
train loss: 0.458569 
test loss: 0.457283
#####################
epoch 116
train loss: 0.456751 
test loss: 0.457214
#####################
epoch 117
train loss: 0.457391 
test loss: 0.457108
#####################
epoch 118
train loss: 0.456739 
test loss: 0.456399
#####################
epoch 119
train loss: 2.303066 
test loss: 0.456748
#####################
epoch 120
train loss: 0.465311 
test loss: 2.375871
#####################
epoch 121
train loss: 2.376911 
test loss: 0.453637
#####################
epoch 122
train loss: 0.455881 
test loss: 0.453774
#####################
epoch 123
train loss: 0.455881 
test loss: 2.306045
#####################
epoch 124
train loss: 2.297686 
test loss: 0.456214
#####################
epoch 125
train loss: 0.456154 
test loss: 0.455710
#####################
epoch 126
train loss: 2.298769 
test loss: 0.455915
#####################
epoch 127
train loss: 0.455390 
test loss: 0.455

D.2 -> Long

In [23]:
'''
Look here -> https://cnvrg.io/pytorch-lstm/
'''
input_dim = 5 
hidden_size = 5
num_layers = 1

device = "cuda" if torch.cuda.is_available() else "cpu"
model = LSTM_0(input_dim , hidden_size , num_layers).to(device)
learning_rate=1e-3

criterion = torch.nn.CrossEntropyLoss()    
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate) 
model = model.double()


num_epochs = 200
for epoch in range(num_epochs): 
    print("epoch " + str(epoch))
    LSTM_train(train_dataloader)
    LSTM_test(test_dataloader)
    print('#####################')

epoch 0
Softmax-processed output -------------
tensor([0.2697, 0.1410, 0.2195, 0.1877, 0.1821], dtype=torch.float64,
       grad_fn=<SoftmaxBackward0>)
Last tensor from last cell state (= unprocessed output) -------------
tensor([ 0.1942, -0.4543, -0.0120, -0.1683, -0.1985], dtype=torch.float64,
       grad_fn=<SelectBackward0>)
###
Softmax-processed output -------------
tensor([0.2480, 0.1578, 0.2327, 0.1946, 0.1668], dtype=torch.float64,
       grad_fn=<SoftmaxBackward0>)
Last tensor from last cell state (= unprocessed output) -------------
tensor([ 0.0935, -0.3587,  0.0299, -0.1488, -0.3029], dtype=torch.float64,
       grad_fn=<SelectBackward0>)
###
Softmax-processed output -------------
tensor([0.2537, 0.1311, 0.2229, 0.1917, 0.2006], dtype=torch.float64,
       grad_fn=<SoftmaxBackward0>)
Last tensor from last cell state (= unprocessed output) -------------
tensor([ 0.0943, -0.5665, -0.0353, -0.1862, -0.1407], dtype=torch.float64,
       grad_fn=<SelectBackward0>)
###
Softmax-pro

Softmax-processed output -------------
tensor([0.2403, 0.1391, 0.3165, 0.1434, 0.1608], dtype=torch.float64,
       grad_fn=<SoftmaxBackward0>)
Last tensor from last cell state (= unprocessed output) -------------
tensor([-0.0606, -0.6073,  0.2148, -0.5770, -0.4626], dtype=torch.float64,
       grad_fn=<SelectBackward0>)
###
Softmax-processed output -------------
tensor([0.2374, 0.1391, 0.3216, 0.1418, 0.1602], dtype=torch.float64,
       grad_fn=<SoftmaxBackward0>)
Last tensor from last cell state (= unprocessed output) -------------
tensor([-0.0772, -0.6121,  0.2262, -0.5929, -0.4709], dtype=torch.float64,
       grad_fn=<SelectBackward0>)
###
Softmax-processed output -------------
tensor([0.2351, 0.1390, 0.3244, 0.1417, 0.1598], dtype=torch.float64,
       grad_fn=<SoftmaxBackward0>)
Last tensor from last cell state (= unprocessed output) -------------
tensor([-0.0893, -0.6149,  0.2328, -0.5956, -0.4756], dtype=torch.float64,
       grad_fn=<SelectBackward0>)
###
Softmax-processed o

tensor([-0.1979, -0.6715,  0.3710, -0.6820, -0.6109], dtype=torch.float64,
       grad_fn=<SelectBackward0>)
###
Softmax-processed output -------------
tensor([0.2127, 0.1330, 0.3832, 0.1312, 0.1399], dtype=torch.float64,
       grad_fn=<SoftmaxBackward0>)
Last tensor from last cell state (= unprocessed output) -------------
tensor([-0.2059, -0.6751,  0.3828, -0.6890, -0.6244], dtype=torch.float64,
       grad_fn=<SelectBackward0>)
###
Softmax-processed output -------------
tensor([0.2099, 0.1331, 0.3854, 0.1313, 0.1403], dtype=torch.float64,
       grad_fn=<SoftmaxBackward0>)
Last tensor from last cell state (= unprocessed output) -------------
tensor([-0.2215, -0.6765,  0.3863, -0.6902, -0.6243], dtype=torch.float64,
       grad_fn=<SelectBackward0>)
###
Softmax-processed output -------------
tensor([0.2062, 0.1330, 0.3915, 0.1307, 0.1386], dtype=torch.float64,
       grad_fn=<SoftmaxBackward0>)
Last tensor from last cell state (= unprocessed output) -------------
tensor([-0.2404, -0

Softmax-processed output -------------
tensor([0.1654, 0.1271, 0.4589, 0.1235, 0.1252], dtype=torch.float64,
       grad_fn=<SoftmaxBackward0>)
Last tensor from last cell state (= unprocessed output) -------------
tensor([-0.4781, -0.7415,  0.5425, -0.7703, -0.7560], dtype=torch.float64,
       grad_fn=<SelectBackward0>)
###
Softmax-processed output -------------
tensor([0.1666, 0.1270, 0.4575, 0.1236, 0.1253], dtype=torch.float64,
       grad_fn=<SoftmaxBackward0>)
Last tensor from last cell state (= unprocessed output) -------------
tensor([-0.4707, -0.7427,  0.5392, -0.7691, -0.7562], dtype=torch.float64,
       grad_fn=<SelectBackward0>)
###
train loss: 0.781975 
test loss: 0.770479
#####################
epoch 9
Softmax-processed output -------------
tensor([0.1585, 0.1451, 0.4439, 0.1245, 0.1280], dtype=torch.float64,
       grad_fn=<SoftmaxBackward0>)
Last tensor from last cell state (= unprocessed output) -------------
tensor([-0.5220, -0.6107,  0.5078, -0.7635, -0.7362], dtype=

Softmax-processed output -------------
tensor([0.1532, 0.1224, 0.4869, 0.1189, 0.1187], dtype=torch.float64,
       grad_fn=<SoftmaxBackward0>)
Last tensor from last cell state (= unprocessed output) -------------
tensor([-0.5523, -0.7767,  0.6043, -0.8052, -0.8070], dtype=torch.float64,
       grad_fn=<SelectBackward0>)
###
Softmax-processed output -------------
tensor([0.1505, 0.1225, 0.4899, 0.1188, 0.1184], dtype=torch.float64,
       grad_fn=<SoftmaxBackward0>)
Last tensor from last cell state (= unprocessed output) -------------
tensor([-0.5702, -0.7764,  0.6100, -0.8070, -0.8103], dtype=torch.float64,
       grad_fn=<SelectBackward0>)
###
Softmax-processed output -------------
tensor([0.1499, 0.1223, 0.4911, 0.1186, 0.1182], dtype=torch.float64,
       grad_fn=<SoftmaxBackward0>)
Last tensor from last cell state (= unprocessed output) -------------
tensor([-0.5741, -0.7778,  0.6127, -0.8083, -0.8119], dtype=torch.float64,
       grad_fn=<SelectBackward0>)
###
Softmax-processed o

...

Softmax-processed output -------------
tensor([0.0925, 0.0875, 0.6438, 0.0874, 0.0888], dtype=torch.float64,
       grad_fn=<SoftmaxBackward0>)
Last tensor from last cell state (= unprocessed output) -------------
tensor([-0.9431, -0.9982,  0.9974, -0.9996, -0.9835], dtype=torch.float64,
       grad_fn=<SelectBackward0>)
###
train loss: 2.421305 
test loss: 0.441455
#####################
epoch 186
Softmax-processed output -------------
tensor([0.0942, 0.0874, 0.6431, 0.0873, 0.0880], dtype=torch.float64,
       grad_fn=<SoftmaxBackward0>)
Last tensor from last cell state (= unprocessed output) -------------
tensor([-0.9237, -0.9980,  0.9975, -0.9998, -0.9913], dtype=torch.float64,
       grad_fn=<SelectBackward0>)
###
Softmax-processed output -------------
tensor([0.0942, 0.0874, 0.6431, 0.0873, 0.0880], dtype=torch.float64,
       grad_fn=<SoftmaxBackward0>)
Last tensor from last cell state (= unprocessed output) -------------
tensor([-0.9239, -0.9980,  0.9975, -0.9998, -0.9913], dtyp

tensor([0.0946, 0.0874, 0.6429, 0.0872, 0.0879], dtype=torch.float64,
       grad_fn=<SoftmaxBackward0>)
Last tensor from last cell state (= unprocessed output) -------------
tensor([-0.9190, -0.9980,  0.9975, -0.9998, -0.9916], dtype=torch.float64,
       grad_fn=<SelectBackward0>)
###
Softmax-processed output -------------
tensor([0.0945, 0.0874, 0.6429, 0.0872, 0.0880], dtype=torch.float64,
       grad_fn=<SoftmaxBackward0>)
Last tensor from last cell state (= unprocessed output) -------------
tensor([-0.9194, -0.9980,  0.9975, -0.9998, -0.9915], dtype=torch.float64,
       grad_fn=<SelectBackward0>)
###
Softmax-processed output -------------
tensor([0.0945, 0.0874, 0.6429, 0.0872, 0.0880], dtype=torch.float64,
       grad_fn=<SoftmaxBackward0>)
Last tensor from last cell state (= unprocessed output) -------------
tensor([-0.9196, -0.9980,  0.9975, -0.9998, -0.9916], dtype=torch.float64,
       grad_fn=<SelectBackward0>)
###
Softmax-processed output -------------
tensor([0.0945, 0.0

tensor([0.0943, 0.0874, 0.6431, 0.0873, 0.0880], dtype=torch.float64,
       grad_fn=<SoftmaxBackward0>)
Last tensor from last cell state (= unprocessed output) -------------
tensor([-0.9223, -0.9981,  0.9977, -0.9998, -0.9918], dtype=torch.float64,
       grad_fn=<SelectBackward0>)
###
Softmax-processed output -------------
tensor([0.0943, 0.0874, 0.6431, 0.0873, 0.0880], dtype=torch.float64,
       grad_fn=<SoftmaxBackward0>)
Last tensor from last cell state (= unprocessed output) -------------
tensor([-0.9225, -0.9981,  0.9977, -0.9998, -0.9919], dtype=torch.float64,
       grad_fn=<SelectBackward0>)
###
Softmax-processed output -------------
tensor([0.0943, 0.0874, 0.6431, 0.0872, 0.0879], dtype=torch.float64,
       grad_fn=<SoftmaxBackward0>)
Last tensor from last cell state (= unprocessed output) -------------
tensor([-0.9225, -0.9981,  0.9978, -0.9998, -0.9920], dtype=torch.float64,
       grad_fn=<SelectBackward0>)
###
Softmax-processed output -------------
tensor([0.0943, 0.0

#####################
epoch 195
Softmax-processed output -------------
tensor([0.0942, 0.0874, 0.6432, 0.0872, 0.0879], dtype=torch.float64,
       grad_fn=<SoftmaxBackward0>)
Last tensor from last cell state (= unprocessed output) -------------
tensor([-0.9235, -0.9982,  0.9979, -0.9999, -0.9920], dtype=torch.float64,
       grad_fn=<SelectBackward0>)
###
Softmax-processed output -------------
tensor([0.0941, 0.0874, 0.6433, 0.0873, 0.0879], dtype=torch.float64,
       grad_fn=<SoftmaxBackward0>)
Last tensor from last cell state (= unprocessed output) -------------
tensor([-0.9239, -0.9982,  0.9979, -0.9999, -0.9919], dtype=torch.float64,
       grad_fn=<SelectBackward0>)
###
Softmax-processed output -------------
tensor([0.0941, 0.0874, 0.6433, 0.0873, 0.0879], dtype=torch.float64,
       grad_fn=<SoftmaxBackward0>)
Last tensor from last cell state (= unprocessed output) -------------
tensor([-0.9241, -0.9982,  0.9979, -0.9999, -0.9919], dtype=torch.float64,
       grad_fn=<SelectBac

tensor([-0.9193, -0.9984,  0.9980, -0.9999, -0.9924], dtype=torch.float64,
       grad_fn=<SelectBackward0>)
###
Softmax-processed output -------------
tensor([0.0921, 0.0876, 0.6439, 0.0874, 0.0889], dtype=torch.float64,
       grad_fn=<SoftmaxBackward0>)
Last tensor from last cell state (= unprocessed output) -------------
tensor([-0.9469, -0.9970,  0.9972, -0.9996, -0.9827], dtype=torch.float64,
       grad_fn=<SelectBackward0>)
###
train loss: 0.440233 
test loss: 0.441696
#####################
epoch 198
Softmax-processed output -------------
tensor([0.0927, 0.0875, 0.6438, 0.0873, 0.0887], dtype=torch.float64,
       grad_fn=<SoftmaxBackward0>)
Last tensor from last cell state (= unprocessed output) -------------
tensor([-0.9407, -0.9985,  0.9978, -0.9997, -0.9843], dtype=torch.float64,
       grad_fn=<SelectBackward0>)
###
Softmax-processed output -------------
tensor([0.0947, 0.0873, 0.6430, 0.0872, 0.0878], dtype=torch.float64,
       grad_fn=<SoftmaxBackward0>)
Last tensor fro

### Y. Thoughts, questions and comments

__The output of the Pytorch LSTM layer is a tuple with two elements. The first element of the tuple is LSTM’s output corresponding to all timesteps (hᵗ : ∀t = 1,2…T) with shape (timesteps, batch, output_features). The second element of the tuple is another tuple with two elements. The first element of this second tuple is the output corresponding to the last timestep (hᵀ). It has the shape (1, batch, output_features). The second element of this second tuple is the cell state corresponding to the last timestep (cᵀ).__ 

### Z. Dirty code

#### About cross entropy loss

In [ ]:
'''
La cross entropy toma un valor sin argmaxear y la etiqueta de la clase correcta 
(no one hot encoding, solo el valor denotado por el indice, i.e. [0-4]). Ahora te toca incorporar esto en el loop.
'''

loss = nn.CrossEntropyLoss()
loss(stacked_fcs[-1], torch.argmax(y[0]))

#### "Checkme" checker

In [11]:
input_size = 5 
hidden_size = 5
num_layers = 1

device = "cuda" if torch.cuda.is_available() else "cpu"
model = LSTM_0(input_size , hidden_size , num_layers).to(device)
learning_rate=1e-3

criterion = torch.nn.CrossEntropyLoss()    
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate) 
model = model.double()

model.train()
for batch , item in enumerate(train_dataloader):
    x , y = item
    x = x.to(device)
    y = y.to(device)
    out, hn, cn, final_out = model(x)
    checkme = (out, (hn, cn), final_out)
    break
    #print("FINAL OUT (Softmax) -------------")
    #print(checkme[2])
    #print("Last element of the last element of 'out' -------------")
    #print(out[-1][-1])
    #print("Last cell state -------------")
    #print(hn[-1][-1])
    #print("################################################" + str(batch))

In [27]:
x.size()

torch.Size([32, 10, 5])

In [28]:
y.size()

torch.Size([32, 5])

In [16]:
x.shape

torch.Size([32, 10, 5])

In [19]:
out.shape

torch.Size([32, 10, 5])

In [20]:
hn.shape

torch.Size([1, 10, 5])

In [21]:
hn

tensor([[[-0.1475,  0.0561,  0.0163, -0.0170, -0.0910],
         [-0.1475,  0.0561,  0.0163, -0.0170, -0.0910],
         [-0.1536,  0.0564,  0.0203, -0.0180, -0.0887],
         [-0.1752,  0.0568,  0.0621, -0.0321, -0.0833],
         [-0.1475,  0.0560,  0.0163, -0.0170, -0.0911],
         [-0.1508,  0.0522,  0.0269, -0.0187, -0.0890],
         [-0.1688,  0.0909,  0.1040, -0.0663, -0.0560],
         [-0.1475,  0.0561,  0.0163, -0.0170, -0.0910],
         [-0.1845,  0.0856,  0.0406, -0.0612, -0.0860],
         [-0.1497,  0.0525,  0.0406, -0.0200, -0.0848]]], dtype=torch.float64,
       grad_fn=<StackBackward0>)

In [13]:
# First batch: 32 inputs ("observations")
len(x)

32

In [18]:
checkme[1][0] # --> last hidden state 

tensor([[[-0.3794, -0.0908,  0.1568,  0.1983,  0.1075],
         [-0.3744, -0.0932,  0.1575,  0.2008,  0.1090],
         [-0.3791, -0.0910,  0.1571,  0.1990,  0.1077],
         [-0.2738, -0.1517,  0.2123,  0.1763,  0.0707],
         [-0.3794, -0.0908,  0.1568,  0.1980,  0.1075],
         [-0.3674, -0.0971,  0.1572,  0.2030,  0.1099],
         [-0.3774, -0.0918,  0.1575,  0.2003,  0.1084],
         [-0.3665, -0.0978,  0.1581,  0.2058,  0.1105],
         [-0.3535, -0.1106,  0.1560,  0.2099,  0.1099],
         [-0.3795, -0.0908,  0.1569,  0.1985,  0.1075]]], dtype=torch.float64,
       grad_fn=<StackBackward0>)

In [20]:
checkme[1][1] # --> last cell state

tensor([[[-0.8294, -0.1876,  0.2826,  0.5188,  0.2271],
         [-0.8136, -0.1928,  0.2838,  0.5255,  0.2300],
         [-0.8284, -0.1880,  0.2831,  0.5210,  0.2275],
         [-0.5729, -0.4017,  0.3055,  0.5759,  0.2020],
         [-0.8294, -0.1875,  0.2825,  0.5182,  0.2270],
         [-0.7919, -0.2015,  0.2829,  0.5304,  0.2319],
         [-0.8230, -0.1897,  0.2838,  0.5246,  0.2287],
         [-0.7894, -0.2031,  0.2843,  0.5383,  0.2330],
         [-0.7384, -0.2281,  0.2779,  0.5450,  0.2316],
         [-0.8296, -0.1876,  0.2827,  0.5195,  0.2271]]], dtype=torch.float64,
       grad_fn=<StackBackward0>)

In [30]:
checkme[0].size() # --> last output (No softmax) (equal to last hidden state by definition)

torch.Size([32, 10, 5])

In [21]:
checkme[2] # --> last output (softmax) -- result of applying softmax to last element of last output

tensor([0.1342, 0.1790, 0.2294, 0.2391, 0.2183], dtype=torch.float64,
       grad_fn=<SoftmaxBackward0>)

In [13]:
# This is what happens when softmax is applied to tensor of shape (10, 5)
torch.softmax(hn[-1], dim=0)

tensor([[0.1010, 0.0994, 0.0980, 0.1011, 0.0994],
        [0.1010, 0.0994, 0.0980, 0.1011, 0.0994],
        [0.1004, 0.0994, 0.0984, 0.1010, 0.0996],
        [0.0982, 0.0995, 0.1026, 0.0996, 0.1002],
        [0.1010, 0.0994, 0.0980, 0.1011, 0.0994],
        [0.1006, 0.0990, 0.0991, 0.1010, 0.0996],
        [0.0988, 0.1029, 0.1070, 0.0963, 0.1030],
        [0.1010, 0.0994, 0.0980, 0.1011, 0.0994],
        [0.0973, 0.1024, 0.1004, 0.0968, 0.0999],
        [0.1007, 0.0991, 0.1004, 0.1008, 0.1000]], dtype=torch.float64,
       grad_fn=<SoftmaxBackward0>)

#### Exploring  the elements of a dataloader object

In [ ]:
'''
Este bloque de código te mete los elementos del dataloader en una lista.
Esta lista está compuesta de tuplas que contienen el identificador del batch como primer elemento,
mientras que como segundo elemento contienen otra tupla con los 32 X e Y que conforman cada batch.
'''
#a = []
#for batch , (item, target) in enumerate(train_dataloader):
#    a.append((batch , (item, target)))

#### Attempt of using the ".load_simulation()" to load the data

In [2]:
params = {'previous_rating_measure': 'mode',
 'review_prior': [1,1,1,1,1],
 'tendency_to_rate': 0.05,
 'simulation_type': 'timeseries',
 'min_reviews_for_herding': 5,
 'num_products': 1400,
 'num_total_marketplace_reviews': 140000,
 'consideration_set_size': 5,
 'one_star_lowest_limit': -1.5,
 'five_star_highest_limit': 1.5,
 'max_bias_5_star': 0.5,
 'num_dist_samples': 10}

simpleloader = marketplace_simulator_class.MarketplaceSimulator(params = params)
simpleloader.load_simulator()

NameError: name 'marketplace_simulator_class' is not defined